# Dự đoán Giá Sản Phẩm

## Ngày 3: Huấn luyện mô hình!

# QUAN TRỌNG vui lòng đọc kỹ!!

Khi bạn chạy các lệnh pip install bên dưới, bạn có thể gặp lỗi từ pip báo về phiên bản fsspec không tương thích.

Bạn nên bỏ qua lỗi đó! Phiên bản fsspec này là phiên bản phù hợp, cần thiết cho HuggingFace.

Nếu bạn hỏi ChatGPT, nó sẽ khuyến nghị bạn pip install phiên bản mới hơn của fsspec. Tuy nhiên, điều đó sẽ gây ra vấn đề; HuggingFace sẽ bị lỗi khi tải bộ dữ liệu với một thông báo lỗi khó hiểu về file systems.

Vì vậy, hãy chạy các lệnh pip install đúng như bên dưới, và nếu thấy lỗi thì cứ bỏ qua nhé!

In [ ]:
# pip installs

!pip install -q datasets requests torch peft bitsandbytes transformers trl accelerate sentencepiece wandb matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# imports
# With much thanks to Islam S. for identifying that there was a missing import!

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
PROJECT_NAME = "pricer"
HF_USER = "kenzytran" # your HF name here!

# Data

DATASET_NAME = f"{HF_USER}/pricer-data"
# Or just use the one I've uploaded
# DATASET_NAME = "ed-donner/pricer-data"
MAX_SEQUENCE_LENGTH = 182

# Run name for saving the model in the hub

RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

# Hyperparameters for QLoRA

LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
LORA_DROPOUT = 0.1
QUANT_4_BIT = True

# Hyperparameters for Training

EPOCHS = 1 # you can do more epochs if you wish, but only 1 is needed - more is probably overkill
BATCH_SIZE = 16 # on an A100 box this can go up to 16
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE = 'cosine'
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"

# Admin config - note that SAVE_STEPS is how often it will upload to the hub
# I've changed this from 5000 to 2000 so that you get more frequent saves

STEPS = 50
SAVE_STEPS = 2000
LOG_TO_WANDB = True

%matplotlib inline

In [ ]:
HUB_MODEL_NAME

'kenzytran/pricer-2025-06-08_11.29.24'

# Thêm về Bộ Tối Ưu (Optimizers)

https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one#optimizer-choice

Phổ biến nhất là Adam hoặc AdamW (Adam với Tối Ưu Trọng Số - Weight Decay).  
Adam đạt được sự hội tụ tốt bằng cách lưu trữ giá trị trung bình động (rolling average) của các gradient trước đó; tuy nhiên, nó tiêu tốn thêm bộ nhớ tương đương với số lượng tham số của mô hình.


### Đăng nhập vào HuggingFace và Weights & Biases

Nếu bạn chưa có tài khoản HuggingFace, hãy truy cập https://huggingface.co để đăng ký và tạo một token.

Sau đó, chọn mục Secrets cho Notebook này bằng cách nhấn vào biểu tượng hình chìa khóa ở bên trái, và thêm một secret mới tên là `HF_TOKEN` với giá trị là token của bạn.

Làm tương tự với weightsandbiases tại https://wandb.ai và thêm một secret tên là `WANDB_API_KEY`

In [ ]:
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# Log in to Weights & Biases
wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

wandb: Currently logged in as: kenzytran (kenzytran-kfsp-company-limited) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

README.md:   0%|          | 0.00/440 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/185M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/914k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# if you wish to reduce the training dataset to 20,000 points instead, then uncomment this line:
# train = train.select(range(20000))

In [ ]:
if LOG_TO_WANDB:
  wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## Bây giờ tải Tokenizer và Mô hình

Mô hình này đã được "lượng tử hóa" – chúng ta đang giảm độ chính xác xuống còn 4 bit.

In [ ]:
# pick the right quantization

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Memory footprint: 5591.5 MB


# Data Collator

Điều quan trọng là trong quá trình huấn luyện, chúng ta phải đảm bảo rằng mô hình không được huấn luyện để dự đoán mô tả sản phẩm, mà chỉ dự đoán giá của sản phẩm.

Chúng ta cần thông báo cho trainer rằng mọi thông tin trước "Price is $" chỉ là ngữ cảnh để mô hình dự đoán token tiếp theo, không cần học thuộc.

Trainer cần dạy mô hình dự đoán token(s) phía sau "Price is $".

Có một cách phức tạp để làm điều này bằng cách thiết lập Mask, nhưng may mắn là HuggingFace đã cung cấp một lớp trợ giúp rất đơn giản để xử lý việc này cho chúng ta.

In [ ]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "Price is $"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# VÀ BÂY GIỜ

## Chúng ta thiết lập cấu hình cho quá trình Huấn luyện

Chúng ta cần tạo 2 đối tượng:

- Một đối tượng LoraConfig với các siêu tham số (hyperparameters) cho LoRA
- Một đối tượng SFTConfig với các tham số tổng thể cho quá trình Huấn luyện

In [ ]:
# # Hyperparameters for QLoRA

# LORA_R = 32
# LORA_ALPHA = 64
# TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]
# LORA_DROPOUT = 0.1
# QUANT_4_BIT = True

# # Hyperparameters for Training

# EPOCHS = 1
# BATCH_SIZE = 16 # on an A100 box this can go up to 16
# GRADIENT_ACCUMULATION_STEPS = 1
# LEARNING_RATE = 1e-4
# LR_SCHEDULER_TYPE = 'cosine'
# WARMUP_RATIO = 0.03
# OPTIMIZER = "paged_adamw_32bit"



# First, specify the configuration parameters for LoRA

lora_parameters = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=TARGET_MODULES,
)

# Next, specify the general configuration parameters for training

train_parameters = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=1,
    eval_strategy="no",
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    optim=OPTIMIZER,
    save_steps=SAVE_STEPS,
    save_total_limit=10,
    logging_steps=STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=WARMUP_RATIO,
    group_by_length=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    report_to="wandb" if LOG_TO_WANDB else None,
    run_name=RUN_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dataset_text_field="text",
    save_strategy="steps",
    hub_strategy="every_save",
    push_to_hub=True,
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True
)

# And now, the Supervised Fine Tuning Trainer will carry out the fine-tuning
# Given these 2 sets of configuration parameters
# The latest version of trl is showing a warning about labels - please ignore this warning
# But let me know if you don't see good training results (loss coming down).

fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=train,
    peft_config=lora_parameters,
    args=train_parameters,
    data_collator=collator
  )

Converting train dataset to ChatML:   0%|          | 0/400000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/400000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/400000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/400000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Ở ô tiếp theo, chúng ta sẽ bắt đầu quá trình fine-tune!

Quá trình này sẽ chạy trong một khoảng thời gian, và sẽ tự động upload lên Hugging Face Hub sau mỗi SAVE_STEPS bước.

Sau một thời gian, Google có thể dừng Colab của bạn. Đối với những ai dùng gói miễn phí, điều này có thể xảy ra bất cứ lúc nào khi Google thiếu tài nguyên. Đối với ai dùng gói trả phí, họ có thể cho bạn chạy tối đa 24 giờ, nhưng cũng không đảm bảo chắc chắn.

Nếu server của bạn bị dừng, bạn có thể làm theo hướng dẫn trong Colab này để tiếp tục từ lần lưu gần nhất:

https://colab.research.google.com/drive/1qGTDVIas_Vwoby4UVi2vwsU0tHXy8OMO#scrollTo=R_O04fKxMMT-

Mình đã lưu Colab này cùng với kết quả cuối cùng ở phần output để bạn tham khảo ví dụ. Lưu ý, mình cần đặt `is_trainable=True` khi load mô hình đã fine-tune.

### Giờ thì, hãy bắt đầu thôi nào!

In [ ]:
# Fine-tune!
fine_tuning.train()

# Push our fine-tuned model to Hugging Face
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


Step,Training Loss
50,2.661300
100,1.970300
150,1.904400
200,1.906800
250,1.916200
300,1.895800
350,1.875600
400,1.894700
450,1.891700
500,1.891900


wandb: Adding directory to artifact (./pricer-2025-06-08_11.29.24/checkpoint-2000)... Done. 0.7s
wandb: Adding directory to artifact (./pricer-2025-06-08_11.29.24/checkpoint-4000)... Done. 0.7s
wandb: Adding directory to artifact (./pricer-2025-06-08_11.29.24/checkpoint-6000)... Done. 0.6s
wandb: Adding directory to artifact (./pricer-2025-06-08_11.29.24/checkpoint-8000)... Done. 0.7s
wandb: Adding directory to artifact (./pricer-2025-06-08_11.29.24/checkpoint-10000)... Done. 0.6s
wandb: Adding directory to artifact (./pricer-2025-06-08_11.29.24/checkpoint-12000)... Done. 0.6s
wandb: Adding directory to artifact (./pricer-2025-06-08_11.29.24/checkpoint-14000)... Done. 0.6s
wandb: Adding directory to artifact (./pricer-2025-06-08_11.29.24/checkpoint-16000)... Done. 0.7s
wandb: Adding directory to artifact (./pricer-2025-06-08_11.29.24/checkpoint-18000)... Done. 0.7s
wandb: Adding directory to artifact (./pricer-2025-06-08_11.29.24/checkpoint-20000)... Done. 0.6s
wandb: Adding directory 

HfHubHTTPError: 504 Server Error: Gateway Time-out for url: https://huggingface.co/api/repos/create

In [ ]:
# Push our fine-tuned model to Hugging Face
fine_tuning.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d4194b85-74ba-4315-ab76-972b91ac249e)') - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3.1-8B.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3.1-8B - will assume that the vocabulary was not modified.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

Saved to the hub: pricer-2025-06-08_11.29.24


In [ ]:
if LOG_TO_WANDB:
  wandb.finish()

train/epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇████
train/grad_norm,▄▆█▄▃▁▂▁▂▂▂▃▃▆▄▃▃▂▃▄▃▂▂▃▃▂▃▄▄▄▂▄▄▄▅▃▅▅▆▃
train/learning_rate,██████▇▇▇▇▇▇▇▆▆▆▆▆▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
train/loss,███▇▆▅▆▅▆▄▅▅▆▅▅▅▅▅▅▄▅▃▃▃▃▃▃▄▃▃▂▂▂▂▂▃▃▁▃▂
train/mean_token_accuracy,▁▂▂▂▂▄▃▃▃▄▃▄▂▅▄▃▃▅▃▄▅▇▅▅▄▅▄▅▅▃▅▅▅▇█▅▇▆▄▅
train/num_tokens,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇███
total_flos,3.231902880983679e+18
train/epoch,1
train/global_step,25000
train/grad_norm,3.59256
